In [1]:
#  *** 1 *** SQL query pro tahani PARCEL 
query_pozemky = """



WITH
-- 1) CTE pro validní záznamy typu 'parcela' bez chyby v GPS_API_info
ValidValuo AS (
    SELECT
        V.*
    FROM
        dbo.Valuo_data AS V
    WHERE
        V.nemovitost = 'parcela'
        AND NOT EXISTS (
            SELECT 1
            FROM dbo.Valuo_data AS V2
            WHERE
                V2.cislo_vkladu = V.cislo_vkladu
                AND (
                    V2.nemovitost <> 'parcela'
                    OR V2.GPS_API_info = 'ERR'
                )
        )
),

-- 2) CTE, které pro každý cislo_vkladu vypočítá celkovou plochu všech parcel
SumArea AS (
    SELECT
        cislo_vkladu,
        SUM(plocha) AS SUM_PLOCHA
    FROM
        ValidValuo
    GROUP BY
        cislo_vkladu
),

-- 3) CTE, které spočítá počet DISTINCT adres (parcel) pro každý cislo_vkladu
ParcelCounts AS (
    SELECT
        V.cislo_vkladu,
        COUNT(DISTINCT V.adresa) AS ParcelCount
    FROM
        ValidValuo AS V
        LEFT JOIN dbo.KN_parcel_data AS K
            ON K.id_valuo = V.id
    GROUP BY
        V.cislo_vkladu
)

-- 4) Hlavní SELECT – ke „validním“ záznamům připojíme SUM_PLOCHA, ParcelCount a další tabulky
SELECT
    V.id                           AS id_valuo,
    U.id                           AS id_up,
    V.cislo_vkladu,
    V.rok,
    V.mesic,
    V.datum_podani,
    V.listina,
    PC.ParcelCount                  AS [#PARCEL],
    SA.SUM_PLOCHA                   AS SUM_PARCEL_RIZENI,
    V.okres,
    V.kat_uzemi                     AS KU_Valuo,
    K.zoning_title                  AS KU_KN,
    K.upper_zoning_id,               --AS kod_ku,
    K.administrativeUnit_title      AS lokalita,
    V.nemovitost,
    --U.*,
    U.POPIS_Z,
	V.typ,
    K.parcel_number,
    V.plocha,
    V.cenovy_udaj,
    -- Výpočet průměrné jednotkové ceny (zaokrouhlené) bez window funkcí:
    CAST(
      ROUND(
        V.cenovy_udaj 
        / NULLIF(SA.SUM_PLOCHA, 0),
        0
      ) 
      AS DECIMAL(38,0)
    )                               AS JC,
    K.gml_id,
    K.areaValue_m2,
    K.beginLifespanVersion,
    K.endLifespanVersion,
    K.geometry,
    K.inspire_localId,
    K.inspire_namespace,
    K.label,
    K.nationalCadastralReference,
    K.refPoint_x,
    K.refPoint_y,
    K.refPoint_lon,
    K.refPoint_lat,
    K.validFrom,
    K.administrativeUnit_href,
    K.administrativeUnit_title     AS adminUnitTitle,
    K.zoning_href,
    K.zoning_title                  AS zoningTitleUP,
    K.id_valuo,
    K.id_UP_FVU_data
FROM
    ValidValuo AS V

    INNER JOIN SumArea AS SA
        ON SA.cislo_vkladu = V.cislo_vkladu

    LEFT JOIN ParcelCounts AS PC
        ON PC.cislo_vkladu = V.cislo_vkladu

    LEFT JOIN dbo.KN_parcel_data AS K
        ON K.id_valuo = V.id

    LEFT JOIN dbo.UP_FVU_data AS U
        ON U.id = K.id_UP_FVU_data

WHERE 1=1
    -- Filtrujeme pouze záznamy, kde průměrná jednotková cena > 0:
    AND CAST(
      ROUND(
        V.cenovy_udaj 
        / NULLIF(SA.SUM_PLOCHA, 0),
        0
      ) 
      AS DECIMAL(38,0)
    ) > 999

    AND CAST(
      ROUND(
        V.cenovy_udaj 
        / NULLIF(SA.SUM_PLOCHA, 0),
        0
      ) 
      AS DECIMAL(38,0)
    ) < 10000


    -- A zároveň jen pro „Hlavní město Praha“
    AND V.okres = 'Hlavní město Praha'
	--AND V.kat_uzemi in ('Písnice', 'Kunratice', 'Libuš', 'Krč')
    -- A nakonec pouze určité kódy v U.POPIS_Z:
    AND (
           U.POPIS_Z LIKE '%DH%'
        OR U.POPIS_Z LIKE '%OP%'
        OR U.POPIS_Z LIKE '%ZMK%'
        OR U.POPIS_Z LIKE '%IZ%'
        OR U.POPIS_Z LIKE 'S[0-9]'
    );





"""

In [6]:
# -*- coding: utf-8 -*-
"""
Jupyter Notebook: Optimalizované zpracování parcel, mapování na ID_VKLADU a prostorové přiřazení k UP polygonům.
"""

# ====================================================================
# 1) Importy a nastavení loggeru
# ====================================================================
import logging
import urllib.parse
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from shapely import wkt
from sqlalchemy import create_engine, text
import folium
from jinja2 import Template
import os
import shutil

# Logger
logger = logging.getLogger("ParcelUP_batch")
logger.setLevel(logging.DEBUG)
fmt = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(fmt)
logger.addHandler(ch)

# ====================================================================
# 2) Vytvoření SQLAlchemy engine
# ====================================================================
params = urllib.parse.quote_plus(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost;"
    "Database=VALUO;"
    "Trusted_Connection=yes;"
)
connection_url = f"mssql+pyodbc:///?odbc_connect={params}"
engine = create_engine(connection_url)

# ====================================================================
# 3) Pomocné funkce pro normalizaci čísla vkladu a mapování ID_VKLADU
# ====================================================================
def normalize_cislo_vkladu(raw: str) -> str:
    """
    Vstup: libovolný text (např. ' 12345 / 2018 '), výstup: '12345/2018'.
    """
    if pd.isna(raw):
        return None
    s = str(raw).strip().replace(" ", "")
    return s

def build_id_vkladu_matrix(list_of_df_with_cislo: list[pd.DataFrame]) -> pd.DataFrame:
    """
    Ze seznamu DataFrame, které mají sloupec 'CISLO_VKLADU', vytvoří tabulku
    ['CISLO_VKLADU', 'ID_VKLADU'], kde ID_VKLADU je pořadové číslo.
    """
    unique_cisla = set()
    for df in list_of_df_with_cislo:
        if 'CISLO_VKLADU' not in df.columns:
            continue
        df['CISLO_VKLADU'] = df['CISLO_VKLADU'].apply(normalize_cislo_vkladu)
        unique_cisla.update(df['CISLO_VKLADU'].dropna().unique())
    unique_cisla = sorted(unique_cisla)
    return pd.DataFrame({'CISLO_VKLADU': unique_cisla, 'ID_VKLADU': range(1, len(unique_cisla)+1)})

def assign_id_vkladu(df: pd.DataFrame, id_matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Přidá ke každému řádku df sloupec 'ID_VKLADU' podle merge s id_matrix.
    """
    df = df.copy()
    df = df.merge(id_matrix, on='CISLO_VKLADU', how='left')
    return df

# ====================================================================
# 4) Pomocné funkce pro převod geometrie
# ====================================================================
def parse_geometry(geom_str: str):
    """
    Vstup: textový řetězec 'x1 y1 x2 y2 ...', výstup: shapely Polygon. 
    Pokud geom_str je None, vrátí None.
    """
    if geom_str is None:
        return None
    try:
        coords = list(map(float, geom_str.split()))
        return Polygon([(coords[i], coords[i+1]) for i in range(0, len(coords), 2)])
    except Exception as e:
        logger.error(f"Chyba při převodu geometry VALUO→POLYGON: '{geom_str[:50]}…': {e}")
        return None

def parse_wkt(wkt_str: str):
    """
    Vstup: WKT text 'POLYGON ((x1 y1, x2 y2, ...))'. Výstup: shapely geometrie.
    """
    if wkt_str is None:
        return None
    try:
        return wkt.loads(wkt_str)
    except Exception as e:
        logger.error(f"Chyba při převodu WKT (UP_FVU_data): '{wkt_str[:50]}…': {e}")
        return None

# ====================================================================
# 5) Načtení „surových“ dat o parcelách a korekce názvu sloupce CISLO_VKLADU
# ====================================================================
logger.info("Načítám data o parcelách (VALUO+KN) z databáze …")
df_parcely_raw = pd.read_sql(query_pozemky, engine)

# Vypsat skutečné názvy sloupců, aby bylo zřejmé, jak se jmenuje číslo vkladu
logger.info(f"Sloupce df_parcely_raw: {df_parcely_raw.columns.tolist()}")

# Pokud sloupec obsahující číslo vkladu neobsahuje přesný název 'CISLO_VKLADU', 
# najdeme jej dynamicky a přejmenujeme:
cislo_cols = [col for col in df_parcely_raw.columns if col.lower() == 'cislo_vkladu']
if not cislo_cols:
    raise KeyError("V tabulce df_parcely_raw nebyl nalezen žádný sloupec odpovídající 'CISLO_VKLADU'.")
# Předpokládáme, že je právě jeden:
orig_cislo_col = cislo_cols[0]
if orig_cislo_col != 'CISLO_VKLADU':
    df_parcely_raw = df_parcely_raw.rename(columns={orig_cislo_col: 'CISLO_VKLADU'})

# Normalizace
df_parcely_raw['CISLO_VKLADU'] = df_parcely_raw['CISLO_VKLADU'].apply(normalize_cislo_vkladu)

# Převod textové geometie na shapely Polygon
df_parcely_raw['geometry'] = df_parcely_raw['geometry'].apply(parse_geometry)

# Vytvoření GeoDataFrame (předpokládané CRS=EPSG:5514, následně převedeme na 4326)
gdf_parcely = gpd.GeoDataFrame(df_parcely_raw, geometry='geometry', crs="EPSG:5514").to_crs(epsg=4326)
logger.info(f"gdf_parcely: CRS={gdf_parcely.crs}, počet={len(gdf_parcely)}")

# ====================================================================
# 6) Načtení UP polygonů a převod WKT
# ====================================================================
sql_up = """
SELECT 
    id, 
    KODFP1_A, 
    geometry 
FROM dbo.UP_FVU_data
WHERE geometry IS NOT NULL
"""
logger.info("Načítám UP polygony z DB …")
df_up_raw = pd.read_sql(sql_up, engine)

# Převod WKT na shapely a vytvoření GeoDataFrame
df_up_raw['up_geometry'] = df_up_raw['geometry'].apply(parse_wkt)
gdf_up = (
    gpd.GeoDataFrame(df_up_raw.drop(columns=['geometry']), 
                     geometry='up_geometry', 
                     crs="EPSG:4326")
    .rename(columns={'up_geometry': 'geometry'})
)
logger.info(f"gdf_up: CRS={gdf_up.crs}, počet={len(gdf_up)}")

# ====================================================================
# 7) Vytvoření centrální mapy CISLO_VKLADU → ID_VKLADU
# ====================================================================
# Seznam všech DF, ve kterých se sloupec CISLO_VKLADU vyskytuje
df_list_for_id = [df_parcely_raw]  # případně zde přidejte další DF, např. df_nemovitosti

id_vkladu_matrix = build_id_vkladu_matrix(df_list_for_id)
logger.info(f"Vytvořeno {len(id_vkladu_matrix)} unikátních ID_VKLADU pro CISLO_VKLADU.")

# ====================================================================
# 8) Připojení ID_VKLADU ke gdf_parcely
# ====================================================================
gdf_parcely = assign_id_vkladu(gdf_parcely, id_vkladu_matrix)
logger.info("Přidán sloupec ID_VKLADU do gdf_parcely.")

# ====================================================================
# 9) Funkce pro prostorové přiřazení UP polygon → parcela (max plocha průniku)
# ====================================================================
def assign_up_with_max_intersection(
    gdf_parcely: gpd.GeoDataFrame,
    gdf_up: gpd.GeoDataFrame,
    engine
):
    """
    Pro každou parcelu z gdf_parcely najde ten UP polygon, kde je maximální plocha průniku (v m²).
    Do tabulky KN_parcel_data uloží id_UP_FVU_data = ID polygonu s maximální průnikem.
    """

    # (1) Ujistíme se, že aktivní geometrie mají název 'geometry'
    if gdf_parcely.geometry.name != 'geometry':
        gdf_parcely = gdf_parcely.set_geometry('geometry')
    if gdf_up.geometry.name != 'geometry':
        gdf_up = gdf_up.set_geometry('geometry')

    # (2) Kontrola shody CRS (musí být oba v EPSG:4326)
    if (gdf_parcely.crs is None 
        or gdf_up.crs is None 
        or (gdf_parcely.crs.to_epsg() != gdf_up.crs.to_epsg())):
        raise ValueError(f"CRS nesouhlasí: parcely={gdf_parcely.crs}, UP={gdf_up.crs}")

    logger.info("Provádím spatial join (intersects) mezi gdf_parcely a gdf_up …")
    joined = gpd.sjoin(
        gdf_parcely,
        gdf_up[['id', 'KODFP1_A', 'geometry']],
        how="inner",
        predicate="intersects"
    )
    logger.info(f"Spatial join vrátil {len(joined)} řádků (včetně vícerých průniků).")

    # (3) Detekce sloupce s ID polygonu v joined: může být 'id' nebo 'id_right'
    if 'id' in joined.columns:
        up_id_col = 'id'
    elif 'id_right' in joined.columns:
        up_id_col = 'id_right'
    else:
        raise KeyError(f"Ve výsledném joined neexistuje ani 'id' ani 'id_right'. Sloupce: {joined.columns.tolist()}")

    # (4) Převod na CRS EPSG:5514 pro výpočet plochy
    gdf_parcely_m = gdf_parcely.to_crs(epsg=5514)
    gdf_up_m      = gdf_up.to_crs(epsg=5514)

    # (5) Udržení původních indexů pro dohledání geometrií
    joined = joined.reset_index().rename(columns={'index': 'orig_index_parcela'})
    joined = joined.rename(columns={'index_right': 'orig_index_up'})

    # (6) Dosažení metrické geometrie parcely a UP v EPSG:5514
    joined['geom_parcela_m'] = joined['orig_index_parcela'].apply(
        lambda idx: gdf_parcely_m.at[idx, 'geometry']
    )
    joined['geom_up_m'] = joined['orig_index_up'].apply(
        lambda idx: gdf_up_m.at[idx, 'geometry']
    )

    # (7) Výpočet intersection a její plochy v m²
    logger.info("Počítám geometrii průniku a plochu (m²).")
    joined['intersection_geom'] = joined.apply(
        lambda row: row['geom_parcela_m'].intersection(row['geom_up_m']),
        axis=1
    )
    joined['intersection_area'] = joined['intersection_geom'].area

    # (8) Pro každou parcelu vybereme řádek s maximální intersection_area
    idx_max = (
        joined
        .loc[joined['intersection_area'] > 0]
        .groupby('parcel_number')['intersection_area']
        .idxmax()
    )
    joined_best = joined.loc[idx_max, ['parcel_number', up_id_col]].copy()
    joined_best = joined_best.rename(columns={up_id_col: 'up_id'})

    logger.info(f"Počet parcel s vybraným up_id: {len(joined_best)}")

    # (9) Hromadný UPDATE do KN_parcel_data.id_UP_FVU_data
    conn = engine.connect()
    trans = conn.begin()
    try:
        for _, row in joined_best.iterrows():
            parc_num = row['parcel_number']
            up_id    = int(row['up_id'])
            update_sql = text("""
                UPDATE dbo.KN_parcel_data
                SET id_UP_FVU_data = :up_id
                WHERE parcel_number = :parc_num
            """)
            conn.execute(update_sql, {"up_id": up_id, "parc_num": parc_num})
        trans.commit()
        logger.info("Hromadné UPDATE do KN_parcel_data úspěšně dokončeno.")
    except Exception as e:
        trans.rollback()
        logger.error(f"Při UPDATE do KN_parcel_data došlo k chybě: {e}")
    finally:
        conn.close()

# ====================================================================
# 10) Spuštění prostorového přiřazení
# ====================================================================
assign_up_with_max_intersection(
    gdf_parcely=gdf_parcely,
    gdf_up=gdf_up,
    engine=engine
)

# ====================================================================
# 11) Agregace parcel per ID_VKLADU
# ====================================================================
agg_parcel_per_vklad = (
    gdf_parcely
    .groupby('ID_VKLADU')
    .agg(
        pocet_parcel      = ('parcel_number', 'nunique'),
        prumer_ocen_cena  = ('OCENOVACI_CENA', 'mean'),
        # Součet plochy v m²: převedeme geometrii na EPSG:5514 a spočítáme area()
        soucet_plocha_m2  = ('geometry', lambda geoms: sum(g.area for g in geoms.to_crs(epsg=5514)))
    )
    .reset_index()
)
logger.info("Vytvořena agregace parcel per vklad.")

# ====================================================================
# 12) Export mapovací a agregované tabulky
# ====================================================================
id_vkladu_matrix.to_csv("id_vkladu_matrix.csv", index=False)
agg_parcel_per_vklad.to_excel("agg_parcel_per_vklad.xlsx", index=False)
logger.info("Export id_vkladu_matrix.csv a agg_parcel_per_vklad.xlsx dokončen.")

# ====================================================================
# 13) Generování datasheetů (HTML + Folium mapa) pro každý CISLO_VKLADU
# ====================================================================
output_dir = "datasheets"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

template_str = """
<!DOCTYPE html>
<html lang="cs">
<head>
  <meta charset="UTF-8"/>
  <title>Datasheet: {{ cislo_vkladu }}</title>
  <style>
    body { font-family: Arial, sans-serif; }
    .container { display: flex; }
    .left-panel  { width: 40%; padding: 10px; }
    .right-panel { width: 60%; padding: 10px; }
    table { border-collapse: collapse; width: 100%; }
    th, td { border: 1px solid #ddd; padding: 8px; text-align: left; }
    th { background-color: #f2f2f2; }
    .iframe-container { width: 100%; height: 500px; border: none; }
  </style>
</head>
<body>
  <div class="container">
    <div class="left-panel">
      <h2>Číslo vkladu: {{ cislo_vkladu }}</h2>
      <table>
        <tr><th>Parcelní číslo</th><th>Ocenění (OC)</th><th>Polygon UP</th></tr>
        {% for row in parcel_rows %}
        <tr>
          <td>{{ row.parcel_number }}</td>
          <td>{{ row.OCENOVACI_CENA }}</td>
          <td>{{ row.KODFP1_A }}</td>
        </tr>
        {% endfor %}
      </table>
    </div>
    <div class="right-panel">
      <iframe class="iframe-container" src="maps/map_{{ cislo_vkladu }}.html"></iframe>
    </div>
  </div>
</body>
</html>
"""
template = Template(template_str)

for _, row in id_vkladu_matrix.iterrows():
    cislo = row['CISLO_VKLADU']
    # Data pro tento vklad
    sub = gdf_parcely[gdf_parcely['CISLO_VKLADU'] == cislo]
    parcel_rows = sub.to_dict(orient='records')
    # Jméno existující mapy (předpokládáme, že maps/map_<cislo>.html již existuje)
    map_filename = f"map_{cislo}.html"
    html_content = template.render(
        cislo_vkladu=cislo,
        parcel_rows=parcel_rows,
        map_filename=map_filename
    )
    safe_name = cislo.replace("/", "_")
    fname = os.path.join(output_dir, f"datasheet_{safe_name}.html")
    with open(fname, "w", encoding="utf-8") as f:
        f.write(html_content)
    logger.info(f"Vygenerován datasheet: {fname}")

logger.info(f"Všechny datasheety jsou ve složce: {output_dir}")


2025-06-02 19:50:29,890 - INFO - Načítám data o parcelách (VALUO+KN) z databáze …
2025-06-02 19:50:29,890 - INFO - Načítám data o parcelách (VALUO+KN) z databáze …
2025-06-02 19:50:29,890 - INFO - Načítám data o parcelách (VALUO+KN) z databáze …
2025-06-02 19:50:30,294 - INFO - Sloupce df_parcely_raw: ['id_valuo', 'id_up', 'cislo_vkladu', 'rok', 'mesic', 'datum_podani', 'listina', '#PARCEL', 'SUM_PARCEL_RIZENI', 'okres', 'KU_Valuo', 'KU_KN', 'upper_zoning_id', 'lokalita', 'nemovitost', 'POPIS_Z', 'typ', 'parcel_number', 'plocha', 'cenovy_udaj', 'JC', 'gml_id', 'areaValue_m2', 'beginLifespanVersion', 'endLifespanVersion', 'geometry', 'inspire_localId', 'inspire_namespace', 'label', 'nationalCadastralReference', 'refPoint_x', 'refPoint_y', 'refPoint_lon', 'refPoint_lat', 'validFrom', 'administrativeUnit_href', 'adminUnitTitle', 'zoning_href', 'zoningTitleUP', 'id_valuo', 'id_UP_FVU_data']
2025-06-02 19:50:30,294 - INFO - Sloupce df_parcely_raw: ['id_valuo', 'id_up', 'cislo_vkladu', 'rok'

AttributeError: The CRS attribute of a GeoDataFrame without an active geometry column is not defined. Use GeoDataFrame.set_geometry to set the active geometry column.